# Packages

In [ ]:
import plotly.express as px
import pandas as pd
from google.cloud import bigquery
import numpy as np
from datetime import time, timedelta, datetime
import math 
import plotly.graph_objects as go
import warnings
warnings.filterwarnings('ignore')
import scipy.stats as st
from tqdm.notebook import tqdm
import matplotlib.pyplot as plt
import seaborn as sns
import os
import ipywidgets
import statsmodels.stats.weightstats as sw
from statsmodels.stats.proportion import test_proportions_2indep
from IPython.display import display, HTML
display(HTML("<style>.container { width:80% !important; }</style>"))
%config InlineBackend.figure_format='retina'
from google.cloud import bigquery
from scipy.stats import norm
from scipy import stats
pd.options.display.float_format = "{:,.2f}".format
from scipy.stats import norm
from scipy import stats
import statsmodels.stats.weightstats as ws
import openpyxl
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import AB_library
from itables import init_notebook_mode
init_notebook_mode(all_interactive=True)
# df["Week"] = df["Date"].dt.to_period("W").dt.to_timestamp()

bigquery_client = bigquery.Client(project='analytics-dev-333113')

def cycle_sql(start, end, query, weeks=False):
    """
    You have to use {date} in your script to add cycle date into this backets
    """
    date_start = datetime.strptime(start, '%Y-%m-%d')
    date_end = datetime.strptime(end, '%Y-%m-%d')

    if weeks == False:
        daterange = [(date_start + timedelta(days=x)).strftime('%Y-%m-%d') for x in range(((date_end-date_start).days)+1)]
    else:
        daterange = [(date_start + timedelta(weeks=x)).strftime('%Y-%m-%d') for x in range(((date_end-date_start).days//7)+1)] # weeks dividing days by 7

    total_df = pd.DataFrame()

    counter = 0

    for date in daterange:
        counter+=1
        print(f"{counter}) Uploading - {date}:", datetime.today().strftime('%Y-%m-%d %H:%M:%S'))
        script = query.format(date = date)
        df_cycle = bigquery_client.query(script).to_dataframe()
        if df_cycle.empty == True:
            print('Dataframe is empty')
        total_df = pd.concat([df_cycle, total_df])
    return total_df

def read_bq(query, project='analytics-dev-333113'):
    client = bigquery.Client(project=project)
    query_job = client.query(query)
    result_df = query_job.to_dataframe()
    return result_df

def display_side_by_side(*args):
    html_str = ''
    for df in args:
        html_str += df.to_html()
    display_html(
        html_str.replace('table','table style="display:inline"'), 
        raw=True
    )

def writing_excel(name:str, dataset1=None, dataset2=None, dataset3=None, dataset4=None):
    with pd.ExcelWriter(f"{name}.xlsx") as writer:
   
    # use to_excel function and specify the sheet_name and index 
    # to store the dataframe in specified sheet

        if dataset1 is not None:
            if dataset2 is not None:
                if dataset3 is not None:
                    if dataset4 is not None:
                        dataset1.to_excel(writer, sheet_name=f"1-{name}", 
                                        #   index=False
                                          )
                        dataset2.to_excel(writer, sheet_name=f"2-{name}", 
                                        #   index=False
                                          )
                        dataset3.to_excel(writer, sheet_name=f"3-{name}", 
                                        #   index=False
                                          )
                        dataset4.to_excel(writer, sheet_name=f"4-{name}", 
                                        #   index=False
                                          )
                    else:
                        dataset1.to_excel(writer, sheet_name=f"1-{name}", 
                                        #   index=False
                                          )
                        dataset2.to_excel(writer, sheet_name=f"2-{name}", 
                                        #   index=False
                                          )
                        dataset3.to_excel(writer, sheet_name=f"3-{name}", 
                                        #   index=False
                                          )
                else:
                    dataset1.to_excel(writer, sheet_name=f"1-{name}", 
                                    #   index=False
                                      )
                    dataset2.to_excel(writer, sheet_name=f"2-{name}", 
                                    #   index=False
                                      )
            else:
                dataset1.to_excel(writer, sheet_name=f"1-{name}", 
                                #   index=False
                                  )

        print('DataFrame is written to Excel File successfully.')

# Descripion

In [2]:
import sys
from PyQt6.QtWidgets import QApplication
import pyqtgraph.opengl as gl
from PyQt6.QtGui import QFont


def main():
    # Создаём приложение
    app = QApplication(sys.argv)

    # Создаём окно
    window = gl.GLViewWidget()
    window.show()
    window.setWindowTitle("3D Phone with Labeled Axes")
    window.setCameraPosition(distance=10)

    # Добавляем красивый фон
    window.setBackgroundColor((30, 30, 30))  # Тёмно-серый фон

    # Добавляем оси
    axes = gl.GLAxisItem()
    axes.setSize(x=5, y=5, z=5)
    window.addItem(axes)

    # Добавляем подписи к осям
    add_axis_labels(window)

    # Добавляем модель телефона
    phone = create_phone()
    window.addItem(phone)

    # Подписываем телефон
    add_label(window, text="Phone", pos=(0, -3, 2), color=(1, 1, 1, 1), size=20)

    # Запускаем главный цикл приложения
    sys.exit(app.exec())


def create_phone():
    """
    Создаёт 3D-модель телефона в виде прямоугольного параллелепипеда.
    """
    # Размеры телефона
    width = 2.0    # Ширина телефона
    height = 4.0   # Высота телефона
    depth = 0.2    # Толщина телефона

    # Создаём вершины телефона
    vertices = [
        [-width / 2, -height / 2, -depth / 2],
        [width / 2, -height / 2, -depth / 2],
        [width / 2, height / 2, -depth / 2],
        [-width / 2, height / 2, -depth / 2],
        [-width / 2, -height / 2, depth / 2],
        [width / 2, -height / 2, depth / 2],
        [width / 2, height / 2, depth / 2],
        [-width / 2, height / 2, depth / 2],
    ]

    # Соединения граней (индексы вершин)
    faces = [
        [0, 1, 2], [0, 2, 3],  # Задняя сторона
        [4, 5, 6], [4, 6, 7],  # Передняя сторона
        [0, 1, 5], [0, 5, 4],  # Нижняя грань
        [2, 3, 7], [2, 7, 6],  # Верхняя грань
        [0, 3, 7], [0, 7, 4],  # Левая грань
        [1, 2, 6], [1, 6, 5],  # Правая грань
    ]

    # Цвет каждой грани (серый)
    colors = [[0.5, 0.5, 0.5, 1.0] for _ in faces]

    # Создаём MeshItem (телефон)
    phone_mesh = gl.GLMeshItem(
        vertexes=vertices,  # Вершины
        faces=faces,        # Грани
        faceColors=colors,  # Цвет граней
        smooth=False,       # Без сглаживания
    )
    phone_mesh.translate(0, 0, 0)  # Размещаем в центре
    return phone_mesh


def add_axis_labels(window):
    """
    Добавляет подписи к осям X, Y, Z.
    """
    # Подпись для оси X
    add_label(window, text="X", pos=(5, 0, 0), color=(1, 0, 0, 1), size=16)

    # Подпись для оси Y
    add_label(window, text="Y", pos=(0, 5, 0), color=(0, 1, 0, 1), size=16)

    # Подпись для оси Z
    add_label(window, text="Z", pos=(0, 0, 5), color=(0, 0, 1, 1), size=16)


def add_label(window, text, pos, color, size):
    """
    Добавляет текстовую метку в 3D-пространство.

    :param window: GLViewWidget
    :param text: Текст метки
    :param pos: Позиция (x, y, z)
    :param color: Цвет текста (R, G, B, A)
    :param size: Размер шрифта
    """
    label = gl.GLTextItem()
    label.setData(
        text=text,
        pos=pos,
        color=color,
        font=QFont("Arial", size)
    )
    window.addItem(label)


if __name__ == "__main__":
    main()

2025-01-27 13:39:41.918 Python[17953:467359] +[IMKClient subclass]: chose IMKClient_Modern


# G-Force analysis